In [32]:
# example of random rotation image augmentation
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
#data handling
import pandas as pd
import numpy as np
import os 
from PIL import Image

import imageio
# Machine learning (non deep learning)
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn import svm
from sklearn.model_selection import GridSearchCV

# Deep learning
import tensorflow as tf
import keras
from keras import layers
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

#misc
import time

In [66]:
len(os.listdir("dataset/split_data/binary/merged/validation/"))

2400

In [67]:
images_list = os.listdir("dataset/split_data/binary/merged/validation")

for i in range(len(images_list)):
    datagen = ImageDataGenerator(rotation_range = 90, width_shift_range=0.25, height_shift_range=0.25, shear_range=0.25, zoom_range=0.25)
    image_path = "dataset/image/"+images_list[i]
    image = np.expand_dims(imageio.imread(image_path), 0)
    save_here = "datagen/test/"
    datagen.fit(image)
    for x, val in zip(datagen.flow(image,                    #image we chose
            save_to_dir=save_here, 
            save_prefix =images_list[i],#this is where we figure out where to save
            save_format='jpg'),range(0)):
        pass


In [66]:
img_reshape_size = (32,32)

data = []
result = []

encoder = OneHotEncoder()
encoder.fit([[0], [1]]) 

# path to tumor files
tumor_image_dir = "dataset/split_data/binary/tumor"
tumor_files = os.listdir(tumor_image_dir)

# path to non tumor files
no_tumor_image_dir = "dataset/split_data/binary/no_tumor"
no_tumor_files = os.listdir(no_tumor_image_dir)

for file in tumor_files:
    temp_file_path = "dataset/split_data/binary/tumor/"+file
    #print("path is " +temp_file_path)
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    data.append(np.array(img))
    result.append(encoder.transform([[1]]).toarray())
        

for file in no_tumor_files:
    temp_file_path = "dataset/split_data/binary/no_tumor/"+file
    #print("path is " +temp_file_path)
    img = Image.open(temp_file_path).convert('L')
    img = img.resize(img_reshape_size)
    img = np.array(img)
    data.append(np.array(img))
    result.append(encoder.transform([[0]]).toarray())

In [67]:
data = np.array(data)
print(data.shape)

result = np.array(result)
result = result.reshape(3000,2)

x_train,x_test,y_train,y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)

(3000, 32, 32)


In [79]:
tumor_labels = pd.read_csv("dataset/label.csv")
for i in range(len(tumor_labels)):
    if (tumor_labels.iloc[i]["label"] == "no_tumor"):
        tumor_labels.iloc[i]["label"] = "no tumor"
    else: 
        tumor_labels.iloc[i]["label"] = "tumor"
tumor_labels

,file_name,label
0,IMAGE_0000.jpg,tumor
1,IMAGE_0001.jpg,no tumor
2,IMAGE_0002.jpg,tumor
3,IMAGE_0003.jpg,tumor
4,IMAGE_0004.jpg,tumor
...,...,...
2995,IMAGE_2995.jpg,no tumor
2996,IMAGE_2996.jpg,tumor
2997,IMAGE_2997.jpg,tumor
2998,IMAGE_2998.jpg,tumor


In [140]:
# create image data augmentation generator
datagen = ImageDataGenerator(validation_split = 0.2, rotation_range = 90, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, rescale=(1/255))

# prepare iterator
data = datagen.flow_from_dataframe(tumor_labels, directory="dataset/image/", x_col="file_name", y_col="label", shuffle=False, color_mode="grayscale")

Found 3000 validated image filenames belonging to 2 classes.


In [141]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(32, 32, 1), padding = 'Same'))
model.add(Conv2D(32, kernel_size=(2, 2),  activation ='relu', padding = 'Same'))


model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))
model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss = "categorical_crossentropy", optimizer='Adamax',  metrics = ['accuracy'])
#print(model.summary())

In [118]:
#param_grid={"C":[0.001,0.01,0.1,1],"gamma":[0.01,0.1,1,10],"kernel":["rbf","sigmoid","poly"]}
param_grid={"C":[0.1],"gamma":[0.1,1],"kernel":["rbf"]}

svc=svm.SVC(probability=True)
model=GridSearchCV(svc,param_grid, verbose = True)

In [142]:
model.fit(data)

InvalidArgumentError:  logits and labels must be broadcastable: logits_size=[2048,2] labels_size=[32,2]
	 [[node categorical_crossentropy/softmax_cross_entropy_with_logits
 (defined at c:\users\camer\appdata\local\programs\python\python39\lib\site-packages\keras\backend.py:5009)
]] [Op:__inference_train_function_5505]

Errors may have originated from an input operation.
Input Source operations connected to node categorical_crossentropy/softmax_cross_entropy_with_logits:
In[0] categorical_crossentropy/softmax_cross_entropy_with_logits/Reshape:	
In[1] categorical_crossentropy/softmax_cross_entropy_with_logits/Reshape_1:

Operation defined at: (most recent call last)
>>>   File "c:\users\camer\appdata\local\programs\python\python39\lib\runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "c:\users\camer\appdata\local\programs\python\python39\lib\runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "c:\us

In [145]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import tensorflow.keras as keras
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [148]:
datagen = ImageDataGenerator(validation_split = 0.2, rotation_range = 90, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2)

train = datagen.flow_from_dataframe(tumor_labels, target_size=(32,32), directory="dataset/image/", x_col="file_name", y_col="label", shuffle=False, color_mode="grayscale", subset="training")
validation = datagen.flow_from_dataframe(tumor_labels, target_size=(32,32), directory="dataset/image/", x_col="file_name", y_col="label", shuffle=False, color_mode="grayscale", subset="validation")

Found 2400 validated image filenames belonging to 2 classes.
Found 600 validated image filenames belonging to 2 classes.


In [176]:
base_model = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.
base_model.trainable = False

In [177]:
inputs = keras.Input(shape=(32, 32, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


In [178]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
model.fit(train, epochs=20, validation_data=validation)


Epoch 1/20


ValueError: in user code:

    File "c:\users\camer\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "c:\users\camer\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\users\camer\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "c:\users\camer\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "c:\users\camer\appdata\local\programs\python\python39\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\users\camer\appdata